In [0]:
pip install openpyxl

In [0]:
from pyspark.sql.functions import current_timestamp, current_date, concat, month, col, lit, when, round, floor, date_format, to_date, countDistinct, avg, sum
from pyspark.sql.types import LongType

import pandas as pd
import openpyxl

In [0]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("example") \
    .config("spark.sql.parquet.mergeSchema", "true") \
    .getOrCreate()

In [0]:
from datetime import datetime, timedelta
import calendar
# from pyspark.sql.functions import  date_sub, expr, dayofweek

# Get the current date
today_date = datetime.now()  

# Get the first date of the current month

first_date_of_current_month = today_date.replace(day=1)

# Get the date of the last passed sunday of current month

latest_sunday_date = today_date - timedelta(days = today_date.weekday()+1)
# month_till_date = today_date - timedelta(days = 1 )

print(first_date_of_current_month.date(), latest_sunday_date.date(), today_date.date())
# latest_past_sunday_date = date_sub(current_date, day_of_week - 1)

In [0]:
from datetime import datetime, date
from datetime import datetime, timedelta
import calendar

def get_month_dates(year, month):
    # First day of the month
    first_date = date(year, month, 1)
    # Last day of the month
    last_date = date(year, month, calendar.monthrange(year, month)[1])
    return first_date, last_date

date_10_days_ago = datetime.today() - timedelta(days=10)
year_10_day_ago = date_10_days_ago.year
month_10_day_ago = date_10_days_ago.month
first_day_10_days_ago, last_day_10_days_ago = get_month_dates(year_10_day_ago, month_10_day_ago)
print("First Date:", first_day_10_days_ago)
print("Last Date:", last_day_10_days_ago)

In [0]:
if today_date.day > 7:
    start = first_date_of_current_month.date()
    end = latest_sunday_date.date()

else:
    start = first_day_10_days_ago
    end = last_day_10_days_ago

start_l = start - timedelta(days=5)


print(start_l, start, end)


In [0]:
# Calculate the date 40 days ago
date_45_days_ago = datetime.today() - timedelta(days=37)

first_day_45_days_ago_l = date_45_days_ago - timedelta(days=3)

year_45_day_ago = date_45_days_ago.year
month_45_day_ago = date_45_days_ago.month
first_day_45_days_ago, last_day_45_days_ago = get_month_dates(year_45_day_ago, month_45_day_ago)

date_6_months_ago = first_day_45_days_ago - timedelta(days=140)
year_6_months_ago = date_6_months_ago.year
month_6_months_ago = date_6_months_ago.month
first_day_6_months_ago, last_day_6_month_ago = get_month_dates(year_6_months_ago, month_6_months_ago)

first_day_6_months_ago_l = first_day_6_months_ago - timedelta(days=5)
last_day_45_days_ago_l = last_day_45_days_ago + timedelta(days=5)

print("First day of the month 6 months ago:", first_day_6_months_ago)
# print("Last day of the month 45 days ago:", last_day_45_days_ago)

# print("First day of the month 45 days ago:", first_day_45_days_ago)
print("Last day of the month 45 days ago:", last_day_45_days_ago)

print("AD start date", first_day_6_months_ago_l)
print("AD end date", last_day_45_days_ago_l)

# print(first_day_45_days_ago_l)


In [0]:
b2c_bird_data = f"""
WITH a AS (
    SELECT DISTINCT
        cl as client,
        wbn,
        COALESCE(mwn, wbn) AS mwn_hudi,
        int_wt_wt AS internal_weight,   
        int_wt_iwt AS dead_wt,
        int_wt_v as volume,
        cs_st as status_type_of_package,
        cs_ss as scan_status,
        int_wt_wt/1000 AS int_wt_kg,
        floor(int_wt_kg / 0.5) * 0.5 AS int_wt_bucket,
        bzn AS bill_zone,
        case when cl = 'FTPL' and pt = 'REPL' then 'S'
        when cl = 'FTPL' and pt != 'REPL' and cs_ss !='DTO' and cnc not in ('Port Blair', 'Leh') then 'S'
        else mot end as mode_of_transport,       
        ctg as prod_cat,  
        CASE
            WHEN pdt = 'B2B' THEN 'B2B'
            WHEN pdt IN ('Heavy', 'Flash_Heavy') THEN 'Heavy'
            ELSE 'B2C'
        END as pdt,
        pt as pkg_type,
        ch_wt_wtr AS chg_wt_rule,
        occ || ' - ' || cnc AS lane,
        date(DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP))) AS closure_date
      
    FROM
        trino_views.hudi_package_wbn_latest_parquet
    WHERE
        ad>= '{first_day_6_months_ago_l}'

        AND cl NOT IN ('HITACHI EXPRESS B2C',
            'BHARATPE EXPRESS',
            'BHARATPESOUNDBOX EXPRESS',
            'BHARATPE SURFACE',
            'DPKAUNIVERSAL EXPRESS',
            'BAJAJ SURFACE',
            'EBOMART B2C',
            'ONE97NEW1 EXPRESS',
            'ONE97NEW SURFACE',
            'AIRTEL PAYMENTS BANK B2C',
            'AIRTEL SURFACE',
            'ONE97 SURFACE',
            'HINIROG1 SURFACE',
            'HINIROGHEALTHTECH SURFACE',
            'HINIROG EXPRESS',
            'BUNDL B2C',
            'Jhakas-Blendjet',
            'ATHER B2C',
            'OLA SURFACE',
            'OLAELECTRIC SURFACE',
            'OLAELECTRICTECHNOLO SURFACE',
            'GMMCO SURFACE',
            'DESIGNFIRST SURFACE',
            'BHARATPE B2C',
            'SVAAS SURFACE',
            'ONE97 B2C')
        AND cnt_cd NOT IN ('LK', 'BD')
        -- AND (intl IS NULL OR intl = 'null' OR NOT intl)
        AND (bzn not IN ('intl', 'INTL') or cl = 'ARAMEX PU SERVICE')
        and cs_ss in ('RTO','DTO','Delivered') 
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) >= '{first_day_6_months_ago}'
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) <= '{last_day_45_days_ago}'
        AND  pdt NOT IN ('B2B', 'Heavy', 'Flash_Heavy')
       
),

b AS (
select * from 
(select *,
ROW_NUMBER() OVER (PARTITION BY wbn ORDER BY action_date DESC) AS row2 
from 
(select waybill_num as wbn, client_name, 
    charged_weight AS billed_wt,
    charged_weight/1000 AS billed_wt_kg,
    floor(billed_wt_kg / 0.5) * 0.5 AS billed_wt_bucket,
    gross_amount , version, 
    from_unixtime(action_date/1000000.0) + interval '330' minute as action_date,
    dense_rank() OVER (PARTITION BY waybill_num ORDER BY version DESC) AS row1
    FROM 
    express_dwh.pq_invoicing_transaction_parquet
            where ad >='{first_day_6_months_ago_l}'
)  WHERE cast(row1 as int) = 1 
) WHERE cast(row2 as int) = 1 
)


select client, mode_of_transport, bill_zone, pkg_type, pdt as product_type, scan_status,   billed_wt_bucket,  date_format(closure_date,'MMMM') as closure_month, month(closure_date) as closure_month_1, year(closure_date) as closure_year,
CAST(count(distinct a.wbn) AS DOUBLE) as wbn_count, sum(gross_amount) as gross_amount_sum,
date(CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP)) as ad,
CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP) as ad_datetime
from a left join b 
on a.wbn = b.wbn
group by client, mode_of_transport, bill_zone, pkg_type, product_type, scan_status,  billed_wt_bucket,  closure_month, closure_month_1, closure_year
order by closure_month_1, closure_year, wbn_count desc 
"""

b2c_last_month_bird_yield_df = spark.sql(b2c_bird_data)


In [0]:
# b2c_last_month_bird_yield_df.write.mode("overwrite").parquet("/dbfs/FileStore/pnl/Bird_last_month_mtd_data_test.parquet")

In [0]:
# b2c_last_month_bird_yield_df  = spark.read.parquet("/dbfs/FileStore/pnl/Bird_last_month_mtd_data_test.parquet")

In [0]:
b2c_last_month_bird_yield_df = b2c_last_month_bird_yield_df.fillna({ 'billed_wt_bucket':-9999})

b2c_last_month_bird_yield_df = b2c_last_month_bird_yield_df.withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)"))


In [0]:
heavy_last_month_query = f"""

WITH a AS (
    SELECT DISTINCT
        date_dlvd,
        date_trunc('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) AS date_dlvdt,
        wbn AS wbn_hudi,
        COALESCE(mwn, wbn) AS mwn_hudi,
        int_wt_wt AS int_wt,
        int_wt_iwt AS dead_wt,
        int_wt_v AS vol_cc,
        int_wt_div AS divisor,
        cs_st,
        int_wt_wt/1000 AS int_wt_kg,
        floor(int_wt_wt / 5000) * 5 AS int_wt_bucket,    
        lrn,
        cl,
        occ,
        cnc,
        bzn ,
        dd_atc AS attempts,
        mot,
        pt,
        cs_ss,
        ch_wt_wtr AS chg_wt_rule,
        int_wt_v / int_wt_div AS vol_wt,
        occ || ' - ' || cnc AS lane,
        date(DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP))) AS closure_date,
        CASE
            WHEN pdt = 'B2B' THEN 'B2B'
            WHEN pdt IN ('Heavy', 'Flash_Heavy') THEN 'Heavy'
            ELSE 'B2C'
        END AS pdt
    FROM
        trino_views.hudi_package_wbn_latest_parquet
    WHERE
        ad >= '{first_day_6_months_ago_l}'
        AND cl NOT IN ('DELHIVERY INTERCHANGE B2B', 'ONE97NEW1 EXPRESS', 'DPKAUNIVERSAL EXPRESS', 'AIRTEL PAYMENTS BANK B2C', 'ONE97NEW SURFACE', 'EBOMART B2C', 'AIRTEL SURFACE', 'BAJAJ SURFACE', 'VOLTAS B2C', 'ATHER B2C', 'BHARATPE SURFACE', 'BHARATPE EXPRESS')
        AND cnt_cd NOT IN ('LK', 'BD')
        --and cl not in ('FEDEX EXPORT' , 'FEDEX IMPORT')
        AND (intl IS NULL OR intl = 'null' OR NOT intl)
        AND bzn NOT IN ('intl', 'INTL')
        AND cs_st = 'DL' 
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) >= '{first_day_6_months_ago}'
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) <= '{last_day_45_days_ago}'
        AND pdt IN ('Heavy') 
), 

b AS (
select * from 
(select *,
ROW_NUMBER() OVER (PARTITION BY wbn_invoicing ORDER BY action_date DESC) AS row2 
from 
(select waybill_num as wbn_invoicing,COALESCE(mwn, waybill_num) AS mwn_invoicing, client_name, 
    charged_weight AS billed_wt,
    charged_weight/1000 AS billed_wt_kg,
    floor(billed_wt_kg / 0.5) * 0.5 AS billed_wt_bucket,
    gross_amount as price, version, 
    from_unixtime(action_date/1000000.0) + interval '330' minute as action_date,
    dense_rank() OVER (PARTITION BY waybill_num ORDER BY version DESC) AS row1
    FROM 
    express_dwh.pq_invoicing_transaction_parquet
            where ad >='{first_day_6_months_ago_l}'
)  WHERE cast(row1 as int) = 1 
) WHERE cast(row2 as int) = 1 
)


SELECT
 cl as client, mot as mode_of_transport, bzn as  bill_zone, pt as pkg_type, pdt as product_type,cs_ss as scan_status, billed_wt_bucket, date_format(closure_date,'MMMM') as closure_month, month(closure_date) as closure_month_1, year(closure_date) as closure_year,
CAST(count(distinct a.mwn_hudi) AS DOUBLE) as wbn_count, sum(price) as gross_amount_sum,
date(CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP)) as ad,
CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP) as ad_datetime
from
 a 
 left join
  b 
on a.wbn_hudi = b.wbn_invoicing

group by client, mode_of_transport, bill_zone, pkg_type, product_type, scan_status, billed_wt_bucket, closure_month, closure_month_1, closure_year
order by closure_month_1, closure_year, wbn_count desc 



"""

heavy_last_month_bird_yield_df = spark.sql(heavy_last_month_query)


In [0]:
heavy_last_month_bird_yield_df = heavy_last_month_bird_yield_df.fillna({ 'billed_wt_bucket':-9999})

heavy_last_month_bird_yield_df = heavy_last_month_bird_yield_df.withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)"))


In [0]:
# from pyspark.sql.functions import col

# heavy_all_last_month_bird_yield_df = heavy_all_last_month_bird_yield_df.withColumn("int_wt_bucket", col("int_wt_bucket").cast("decimal(10,4)")).withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)"))


In [0]:
ftpl_data = f"""
WITH a AS (
    SELECT DISTINCT
        cl as client,
        wbn,
        int_wt_wt AS internal_weight,   
        int_wt_iwt AS dead_wt,
        int_wt_v as volume,
        cs_st as status_type_of_package,
        cs_ss as scan_status,
        int_wt_wt/1000 AS int_wt_kg,
        floor(int_wt_kg / 0.15) * 0.15 AS int_wt_bucket,
        bzn AS bill_zone,
        case when cl = 'FTPL' and pt = 'REPL' then 'S'
        when cl = 'FTPL' and pt != 'REPL' and cs_ss !='DTO' and cnc not in ('Port Blair', 'Leh') then 'S'
        else mot end as mode_of_transport,       
        ctg as prod_cat,  
        CASE
            WHEN pdt = 'B2B' THEN 'B2B'
            WHEN pdt IN ('Heavy', 'Flash_Heavy') THEN 'Heavy'
            ELSE 'B2C'
        END as prod_desc,
        pt as pkg_type,
        ch_wt_wtr AS chg_wt_rule,
        occ || ' - ' || cnc AS lane,
        date(DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP))) AS closure_date
      
    FROM
        trino_views.hudi_package_wbn_latest_parquet
    WHERE
        ad>= '{start_l}'
        and cl = 'FTPL SURFACE'
        AND cnt_cd NOT IN ('LK', 'BD')
        AND (bzn not IN ('intl', 'INTL') or cl = 'ARAMEX PU SERVICE')
        and cs_ss in ('RTO','DTO','Delivered')
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) >= '{start}'
        AND DATE_TRUNC('day', CAST(date_dlvdt + INTERVAL '330' MINUTE AS TIMESTAMP)) <= '{end}'
        AND  pdt NOT IN ('B2B', 'Heavy', 'Flash_Heavy')
       
), 
b AS (
select * from 
(select *,
ROW_NUMBER() OVER (PARTITION BY wbn ORDER BY action_date DESC) AS row2 
from 
(select waybill_num as wbn, client_name, 
    charged_weight AS billed_wt,
    charged_weight/1000 AS billed_wt_kg,
    floor(billed_wt_kg / 0.5) * 0.5 AS billed_wt_bucket,
    gross_amount , version, 
    from_unixtime(action_date/1000000.0) + interval '330' minute as action_date,
    dense_rank() OVER (PARTITION BY waybill_num ORDER BY version DESC) AS row1
    FROM 
    express_dwh.pq_invoicing_transaction_parquet
            where ad >='{start_l}'
)  WHERE cast(row1 as int) = 1 
) WHERE cast(row2 as int) = 1 
)


select client, mode_of_transport, bill_zone, pkg_type, prod_desc as product_type, scan_status,  billed_wt_bucket, date_format(closure_date,'MMMM') as closure_month, month(closure_date) as closure_month_1, year(closure_date) as closure_year,
CAST(count(distinct a.wbn) AS DOUBLE) as wbn_count, sum(gross_amount) as gross_amount_sum,
date(CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP)) as ad,
CAST(now() + INTERVAL '330' MINUTE AS TIMESTAMP) as ad_datetime
from a left join b 
on a.wbn = b.wbn
group by client, mode_of_transport, bill_zone, pkg_type, product_type, scan_status,  billed_wt_bucket, closure_month, closure_month_1, closure_year
order by closure_month_1, closure_year, wbn_count desc 


"""

ftpl_data_df = spark.sql(ftpl_data)


In [0]:
ftpl_data_df = ftpl_data_df.fillna({ 'billed_wt_bucket':-9999})

ftpl_data_df = ftpl_data_df.withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)"))


In [0]:
print(ftpl_data_df.dtypes == b2c_last_month_bird_yield_df.dtypes)
print(heavy_last_month_bird_yield_df.dtypes == ftpl_data_df.dtypes) 
print(b2c_last_month_bird_yield_df.dtypes == heavy_last_month_bird_yield_df.dtypes) 

In [0]:
b2c_ftpl_bird_data_joined = b2c_last_month_bird_yield_df.union(ftpl_data_df) 
b2c_heavy_bird_data_joined = b2c_ftpl_bird_data_joined.union(heavy_last_month_bird_yield_df)

In [0]:
b2c_heavy_bird_data_joined.write.mode("overwrite").parquet("/dbfs/FileStore/pnl/Bird_last_6_month_yield_granular_data.parquet")

In [0]:
bird_data_df  = spark.read.parquet("/dbfs/FileStore/pnl/Bird_last_6_month_yield_granular_data.parquet")

In [0]:
# Create int_wt_bkt and closure date column
from pyspark.sql.functions import col, concat_ws, to_date, add_months, last_day

bird_data_df = bird_data_df.withColumn('int_wt_bucket',col('billed_wt_bucket')).withColumn("first_day", to_date(concat_ws("-", col("closure_year"), col("closure_month_1"), lit("01")), "yyyy-M-d")).withColumn("closure_date", last_day(col("first_day"))).drop("first_day","closure_month_1","closure_year").select('client', 'mode_of_transport', 'bill_zone', 'pkg_type', 'product_type', 'scan_status',
 'int_wt_bucket', 'billed_wt_bucket', 'closure_date', 'closure_month', 'wbn_count', 'gross_amount_sum', 'ad', 'ad_datetime')


In [0]:
# Selecting FTPL Surface current month data
ftpl_sur_data = bird_data_df.filter((col('client') == 'FTPL SURFACE') & (col('closure_date') >= start)).withColumn('closure_date', lit(end))
# Selcting b2c and heavy past 6 month bird data
bird_data_df = bird_data_df.filter((col('closure_date')>=first_day_6_months_ago) & (col('closure_date')<=last_day_45_days_ago))
display(ftpl_sur_data)

In [0]:
display(bird_data_df.count())

In [0]:
display(bird_data_df.filter((col('client')=='AMAZONINDIA')).groupBy('product_type','closure_month','closure_date','client').agg(sum('wbn_count'), sum('gross_amount_sum')))
#  AMAZONINDIA FLIPKART - E2E SURFACE


In [0]:
display(bird_data_df.select('product_type','closure_month','closure_date').distinct())

In [0]:
b2c_estimated_mtd = spark.read.parquet("/dbfs/FileStore/pnl/b2c_yield_revenue_estimate.parquet")
# change path to b2c_yield_revenue_estimate next time
b2c_estimated_mtd = b2c_estimated_mtd.withColumn('closure_date', lit(end))

In [0]:
display(b2c_estimated_mtd)

In [0]:
%sql
-- select * from gsheet_s3_integrations.data_analytics_client_to_aop_mapping
select * from gsheet_s3_integrations.data_analytics_discount_adjustment_and_fallback
-- select * from gsheet_s3_integrations.data_analytics_discount_and_provisions_b2c_clients
;

In [0]:
discount_and_fallback = _sqldf

In [0]:
%sql
select * from gsheet_s3_integrations.data_analytics_discount_and_provisions_b2c_clients;

In [0]:
b2c_client_dis_prov = _sqldf

In [0]:
fedex_ref = discount_and_fallback.filter((col('category')=='Closure Factor')).withColumnRenamed('value', 'perc').withColumnRenamed('client_name', 'cl').select('cl','perc')

fedex_ref = fedex_ref.join(b2c_client_dis_prov, fedex_ref['cl']== b2c_client_dis_prov['external_id'], how='left')
fedex_ref = fedex_ref[['cl' ,  'perc' ,  'gross_yield' ]]

display(fedex_ref)

In [0]:
# display(b2c_estimated_mtd_fedex.groupBy('cl').agg({'count_new':'sum'}))
# b2c_estimated_mtd.columns

In [0]:
b2c_estimated_mtd_fedex = b2c_estimated_mtd.filter(col('cl').isin('FEDEX EXPORT','FEDEX IMPORT','TNT EXPORT','TNT IMPORT') & (col('closure_date') >= start) & (col('closure_date') <= end))

# display(b2c_estimated_mtd_fedex.groupBy('cl').agg({'count_new':'sum'}))

b2c_estimated_mtd_fedex2 = b2c_estimated_mtd_fedex.join(fedex_ref, on='cl', how='left')
# display(b2c_estimated_mtd_fedex2)

b2c_estimated_mtd_fedex2 = b2c_estimated_mtd_fedex2.withColumn('wbn_adjusted',  col('perc') * col('count_new')).withColumn('avg_price_adjusted', col('wbn_adjusted') * col('gross_yield'))

b2c_estimated_mtd_fedex2 = b2c_estimated_mtd_fedex2.drop(*['perc','gross_yield','count_new','avg_price_total'])
# display(b2c_estimated_mtd_fedex2)

b2c_estimated_mtd_fedex2 = b2c_estimated_mtd_fedex2.withColumnRenamed('wbn_adjusted', 'count_new').withColumn('price_total', col('avg_price_adjusted')).withColumnRenamed('avg_price_adjusted', 'avg_price_total').select('cl', 'closure_date', 'pdt','mot','zone','pt','cs_ss','wt_bucket_value', 'count_new' ,'avg_price_total', 'price_total', 'update_date', 'ad')

# .withColumn("count_new", round(col("count_new")).cast('int'))

display(b2c_estimated_mtd_fedex2)

In [0]:
display(b2c_estimated_mtd_fedex2.groupBy('cl').agg({'count_new':'sum'}))

In [0]:
b2c_estimated_mtd_filtered = b2c_estimated_mtd.select('cl',
 'closure_date',
 'pdt',
 'mot',
 'zone',
 'pt',
 'cs_ss',
 'wt_bucket_value',
 'count_new',
 'avg_price_total',
 'billed_wt_estimated',
 'update_date',
 'ad').filter((col('closure_date') >= start) & (col('closure_date') <= end) & (~col('cl').isin('FEDEX EXPORT','FEDEX IMPORT','TNT EXPORT','TNT IMPORT','FTPL SURFACE')))

b2c_estimated_mtd_joined = b2c_estimated_mtd_filtered.union(b2c_estimated_mtd_fedex2)

b2c_estimated_mtd_joined = b2c_estimated_mtd_joined.withColumn('billed_wt_bucket', col('wt_bucket_value')).withColumn('closure_month', date_format(col('closure_date'), "MMMM")).withColumn('count_new', col('count_new')).withColumn('product_type',lit('B2C')).withColumnRenamed('count_new', 'wbn_count').withColumnRenamed('mot', 'mode_of_transport').withColumnRenamed('cs_ss', 'scan_status').withColumnRenamed('pt', 'pkg_type').withColumnRenamed('zone', 'bill_zone').withColumnRenamed('wt_bucket_value', 'int_wt_bucket').withColumnRenamed('avg_price_total', 'gross_amount_sum').withColumnRenamed('cl', 'client').select('client', 'mode_of_transport', 'bill_zone', 'pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month','wbn_count','gross_amount_sum')

b2c_estimated_mtd_joined = b2c_estimated_mtd_joined.groupBy('client', 'mode_of_transport', 'bill_zone', 'pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month').agg(sum('wbn_count').alias('wbn_count'), sum('gross_amount_sum').alias('gross_amount_sum')).withColumn("ad", current_date()).withColumn("ad_datetime", current_timestamp())

print(b2c_estimated_mtd_joined.columns == ftpl_sur_data.columns)
ftpl_sur_data = ftpl_sur_data.filter((col('closure_date') >= start) & (col('closure_date') <= end))

b2c_estimated_mtd_joined_2 = b2c_estimated_mtd_joined.union(ftpl_sur_data)

print(b2c_estimated_mtd_joined_2.count()== b2c_estimated_mtd_joined.count() + ftpl_sur_data.count())

b2c_estimated_mtd_joined_2 = b2c_estimated_mtd_joined_2.withColumn('product_type',lit('B2C')).select('client', 'mode_of_transport', 'bill_zone', 'pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month','wbn_count','gross_amount_sum','ad','ad_datetime').filter((col('closure_date') >= start) & (col('closure_date') <= end))

# display(b2c_estimated_mtd_joined_2.filter(col('client')=='FTPL SURFACE'))

b2c_estimated_mtd_joined_2 = b2c_estimated_mtd_joined_2.fillna({'int_wt_bucket':-9999, 'billed_wt_bucket':-9999})

b2c_estimated_mtd_joined_2 = b2c_estimated_mtd_joined_2.withColumn("int_wt_bucket", col("int_wt_bucket").cast("decimal(10,4)")).withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)"))
# .withColumn("wbn_count", col("wbn_count").cast(LongType()))

display(b2c_estimated_mtd_joined_2.head(50))

In [0]:
# Qc = b2c_estimated_mtd_joined_2.filter(col('client').isin('FEDEX EXPORT','FEDEX IMPORT','TNT EXPORT','TNT IMPORT'))
# display(Qc.groupBy('client').agg({'wbn_count':'sum'}))

In [0]:
heavy_estimate_mtd = spark.read.parquet('/dbfs/FileStore/vg_revenue_estimate_tbd/Heavy_yield_mtd_estimate_all_columns.parquet')
display(heavy_estimate_mtd.head(10))

In [0]:
heavy_estimate_mtd = heavy_estimate_mtd.withColumn('int_wt_kg', col('int_wt')/1000).withColumn('int_wt_bucket', floor(col('int_wt')/5000)*5).withColumn('billed_wt_bucket', col('int_wt_bucket')).withColumn('closure_month', date_format(col('closure_date'), "MMMM")).withColumnRenamed('zone','bill_zone').withColumnRenamed('pt','pkg_type').withColumnRenamed('mot','mode_of_transport').withColumnRenamed('cs_ss','scan_status').withColumnRenamed('cl','client').withColumn('product_type',lit('Heavy')).withColumn('closure_date', lit(end))

heavy_estimate_grouped = heavy_estimate_mtd.groupBy('client','mode_of_transport','bill_zone','pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month').agg(countDistinct('mwn_hudi').alias('wbn_count'),sum('avg_price').alias('gross_amount_sum'))

heavy_estimate_final_df = heavy_estimate_grouped.withColumn("ad_datetime", current_timestamp()).withColumn("ad", current_date()).withColumn("int_wt_bucket", col("int_wt_bucket").cast("decimal(10,4)")).withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)")).withColumn("wbn_count", col("wbn_count").cast("double")).select('client', 'mode_of_transport', 'bill_zone', 'pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month','wbn_count','gross_amount_sum','ad','ad_datetime')

display(heavy_estimate_final_df.head(10))

In [0]:
# heavy_all_estimate_mtd = heavy_estimate_mtd.withColumn('int_wt_bucket', lit(0.00)).withColumn('billed_wt_bucket', lit(0.00)).withColumn('closure_month', date_format(col('closure_date'), "MMMM")).withColumn('bill_zone',lit('All')).withColumn('pkg_type',lit('All')).withColumn('mode_of_transport',lit('All')).withColumn('scan_status',lit('All')).withColumnRenamed('cl','client').withColumn('product_type',lit('Heavy_Estimate_All'))

# heavy_all_estimate_grouped = heavy_all_estimate_mtd.groupBy('client','mode_of_transport','bill_zone','pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month').agg(countDistinct('mwn_hudi').alias('wbn_count'),sum('avg_price').alias('gross_amount_sum'))

# heavy_all_estimate_final_df = heavy_all_estimate_grouped.withColumn("ad_datetime", current_timestamp()).withColumn("ad", current_date()).withColumn("int_wt_bucket", col("int_wt_bucket").cast("decimal(10,4)")).withColumn("billed_wt_bucket", col("billed_wt_bucket").cast("decimal(10,4)")).withColumn("wbn_count", col("wbn_count").cast("double")).select('client', 'mode_of_transport', 'bill_zone', 'pkg_type','product_type','scan_status','int_wt_bucket','billed_wt_bucket','closure_date','closure_month','wbn_count','gross_amount_sum','ad','ad_datetime')

# display(heavy_all_estimate_final_df.head(10))

In [0]:
bird_data_df = bird_data_df.withColumn("ad_datetime", current_timestamp()).withColumn("ad", current_date())
display(bird_data_df.select('ad').distinct())

In [0]:
print(b2c_estimated_mtd_joined_2.columns == heavy_estimate_final_df.columns)
print(b2c_estimated_mtd_joined_2.dtypes == heavy_estimate_final_df.dtypes)

print(b2c_estimated_mtd_joined_2.columns == bird_data_df.columns)
print(b2c_estimated_mtd_joined_2.dtypes == bird_data_df.dtypes)

# print(b2c_estimated_mtd_joined_2.columns == heavy_all_estimate_final_df.columns)
# print(b2c_estimated_mtd_joined_2.dtypes == heavy_all_estimate_final_df.dtypes)

In [0]:
Bird_and_b2c_data_df = bird_data_df.union(b2c_estimated_mtd_joined_2)  
# Bird_b2c_hvy_all_data_df = Bird_and_b2c_data_df.union(heavy_all_estimate_final_df)  
Bird_b2c_heavy_data_df = Bird_and_b2c_data_df.union(heavy_estimate_final_df)    

In [0]:
display(Bird_b2c_heavy_data_df.filter(col('client') == 'SUPERSTORE BY NYKAA').groupBy('client','product_type','closure_month','closure_date').agg({'wbn_count':'sum','gross_amount_sum':'sum'}))

In [0]:
display(Bird_b2c_heavy_data_df.groupBy('ad','product_type','closure_month','closure_date').agg(sum(col('wbn_count')).alias('total_wbn'),sum(col('gross_amount_sum')).alias('gross_amount')).withColumn('gross_yield', round(col('gross_amount')/col('total_wbn'),2)))

In [0]:
# print(bird_data_df.count(),Estimated_mtd_joined.count(), Bird_and_estimated_data_df.count())
print(Bird_b2c_heavy_data_df.count() == bird_data_df.count() + b2c_estimated_mtd_joined_2.count() + heavy_estimate_final_df.count())
print(Bird_b2c_heavy_data_df.count()) 
Bird_b2c_heavy_data_df.dtypes


In [0]:
display(Bird_b2c_heavy_data_df)

In [0]:
%python
# Writing data to S3 
Bird_b2c_heavy_data_df.write.mode("overwrite").partitionBy("ad").parquet("s3://abis3/Sheetal/Monthly_yield_estimated_b2c_heavy_bird_data")

In [0]:
# Delete data from S3 bucket
# dbutils.fs.rm('s3://abis3/Sheetal/Monthly_yield_estimated_b2c_heavy_bird_data/ad=2024-11-14', True)

In [0]:
# df = spark.read.parquet("s3://abis3/Sheetal/Monthly_yield_summary/")

In [0]:
%sql
refresh table data_analytics.bi_revenue_estimate_and_bird_data 

In [0]:
%sql

SELECT ad, count(*) 

    FROM data_analytics.bi_revenue_estimate_and_bird_data 
    WHERE
        ad >= (select max(ad) from data_analytics.bi_revenue_estimate_and_bird_data)
        group by 1;


In [0]:
%python
# Writing data to DBFS
# monthly_yield_df.write.mode("overwrite").parquet("/dbfs/mnt/abis3-mount/sheetal/ss_monthly_yield.parquet")
# monthyl_yield_df.write.mode("overwrite").csv("/dbfs/mnt/abis3-mount/sheetal/ss_monthly_yield.csv")
